<a href="https://colab.research.google.com/github/bhumong/ai-bootcamp/blob/main/project/03B_pytorch_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Deep Learning - Pytorch

**Instructions for Students:**

Please carefully follow these steps to complete and submit your project:

1. **Make a copy of the Project**: Please make a copy of this project either to your own Google Drive or download locally. Work on the copy of the project. The master project is **Read-Only**, meaning you can edit, but it will not be saved when you close the master project. To avoid total loss of your work, remember to make a copy.

2. **Completing the Project**: You are required to work on and complete all tasks in the provided project. Be disciplined and ensure that you thoroughly engage with each task.
   
3. **Creating a Google Drive Folder**: Each of you must create a new folder on your Google Drive. This will be the repository for all your completed project files, aiding you in keeping your work organized and accessible.
   
4. **Uploading Completed Project**: Upon completion of your project, make sure to upload all necessary files, involving codes, reports, and related documents into the created Google Drive folder. Save this link in the 'Student Identity' section and also provide it as the last parameter in the `submit` function that has been provided.
   
5. **Sharing Folder Link**: You're required to share the link to your project Google Drive folder. This is crucial for the submission and evaluation of your project.
   
6. **Setting Permission to Public**: Please make sure your Google Drive folder is set to public. This allows your instructor to access your solutions and assess your work correctly.

Adhering to these procedures will facilitate a smooth project evaluation process for you and the reviewers.

## Project Description

The Deep Learning Projects are divided into two parts, the first is the Calculations worth 30% and the second one is Pytorch Project worth 70% in this notebook.

The two projects will help you gain experience to learn about Deep Learning in detail.

In this project, you will use what you learn to create your own Deep Learning model. We'll use a variety of datasets, each with different data types such as images, text, and numerical/categorical data. Your task is to choose one of these datasets and build a deep learning model upon it.


## Datasets

Choose one from the following datasets for your project:

1. **Digits Dataset**: A simplified version of MNIST containing 8x8 images of hand-written digits. [Dataset Link](https://scikit-learn.org/stable/auto_examples/datasets/plot_digits_last_image.html)
2. **Dogs vs. Cats Dataset from Kaggle**: A sizable dataset of 25,000 images with equal representation of dogs and cats. [Dataset Link](https://www.kaggle.com/c/dogs-vs-cats)
3. **Breast Cancer Wisconsin (Diagnostic) Dataset**: Computed features from a digitized image of a fine needle aspirate (FNA) of a breast mass. [Dataset Link](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)
4. **Spam Text Message Classification Dataset**: A collection of labeled SMS messages, categorized as "spam" or "ham". [Dataset Link](https://www.kaggle.com/uciml/sms-spam-collection-dataset)
5. **German Credit Risk Dataset**: People's data categorized by various attributes to predict credit risk. [Dataset Link](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29)



## Grading Criteria

Your work will be evaluated based on both accuracy and loss value:

100: The model has an accuracy of more than 80% and a Loss Value of less than 0.2. This model is excellent and demonstrates a strong understanding of the task.

90: The model has an accuracy between 70% - 79% and a Loss Value between 0.2 - 0.3. This model is very good, with some room for improvement.

80: The model has an accuracy between 60% - 69% and a Loss Value between 0.3 - 0.4. This model is fairly good but needs improvement in balancing accuracy and loss value.

70: The model has an accuracy between 50% - 59% and a Loss Value between 0.4 - 0.5. This model is below average and needs significant improvement.

60 or below: The model has an accuracy of less than 50% or a Loss Value of more than 0.5, or the student did not submit the accuracy and Loss Value. This model is poor and needs considerable improvement.

Rmember to make a copy of this notebook in your Google Drive and work in your own copy.

Let's start your deep learning journey! Choose your dataset and delve into the project! Happy modeling!

## Student Identity

In [1]:
# @title #### Student Identity
student_id = "REA6HCHTW" # @param {type:"string"}
name = "samuel limong" # @param {type:"string"}
drive_link = "https://drive.google.com/drive/folders/1K6U077XHpD5FwrN--cFBJH6g6G2XW2rP?usp=sharing"  # @param {type:"string"}

assignment_id = "00_pytorch_project"

# Import grader package
!pip install rggrader
from rggrader import submit, submit_image


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Project Structure

Your project should be organized into five main sections.

### 1. Package and Module Installation

First, let's pool all package and module that you'll need in the installation section below.

In [2]:
!kaggle datasets download uciml/sms-spam-collection-dataset

Dataset URL: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
License(s): unknown
sms-spam-collection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip sms-spam-collection-dataset.zip

Archive:  sms-spam-collection-dataset.zip
  inflating: spam.csv                


In [5]:
!pip install pandas torch scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


### 2. Data Loading and Preprocessing

Load the chosen dataset and preprocess it for deep learning.

In [6]:
# Write your preprocessing here
import pandas as pd

data = pd.read_csv("./spam.csv", header=None, encoding='latin-1')
data = data[[0, 1]]
data.columns = ['label', 'text']
data.drop(0, inplace= True)

data['label'] = data['label'].map({"ham": 0, "spam": 1})
data

,label,text
1,0,"Go until jurong point, crazy.. Available only ..."
2,0,Ok lar... Joking wif u oni...
3,1,Free entry in 2 a wkly comp to win FA Cup fina...
4,0,U dun say so early hor... U c already then say...
5,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5568,1,This is the 2nd time we have tried 2 contact u...
5569,0,Will Ì_ b going to esplanade fr home?
5570,0,"Pity, * was in mood for that. So...any other s..."
5571,0,The guy did some bitching but I acted like i'd...


In [7]:
def tokenize(text):
    return text.lower().split()

data['tokens'] = data['text'].apply(tokenize)
data

,label,text,tokens
1,0,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point,, crazy.., available..."
2,0,Ok lar... Joking wif u oni...,"[ok, lar..., joking, wif, u, oni...]"
3,1,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
4,0,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor..., u, c, already..."
5,0,"Nah I don't think he goes to usf, he lives aro...","[nah, i, don't, think, he, goes, to, usf,, he,..."
...,...,...,...
5568,1,This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,..."
5569,0,Will Ì_ b going to esplanade fr home?,"[will, ì_, b, going, to, esplanade, fr, home?]"
5570,0,"Pity, * was in mood for that. So...any other s...","[pity,, *, was, in, mood, for, that., so...any..."
5571,0,The guy did some bitching but I acted like i'd...,"[the, guy, did, some, bitching, but, i, acted,..."


In [8]:

from collections import Counter

counter = Counter()
for tokens in data['tokens']:
    counter.update(tokens)
counter

Counter({'to': 2226,
         'i': 2208,
         'you': 1917,
         'a': 1419,
         'the': 1317,
         'u': 991,
         'and': 967,
         'is': 862,
         'in': 855,
         'my': 751,
         'for': 697,
         'your': 676,
         'of': 613,
         'me': 605,
         'have': 568,
         'call': 557,
         'on': 520,
         'are': 485,
         'that': 468,
         'it': 465,
         '2': 453,
         'so': 423,
         'but': 420,
         'or': 414,
         'not': 404,
         'at': 398,
         'can': 385,
         'ur': 379,
         'if': 378,
         'with': 377,
         "i'm": 377,
         'be': 375,
         'get': 375,
         'will': 374,
         'do': 365,
         'just': 363,
         'we': 345,
         'this': 312,
         'when': 282,
         'from': 277,
         '&lt;#&gt;': 276,
         'go': 262,
         'up': 262,
         'all': 259,
         'no': 257,
         'how': 253,
         'what': 250,
         '4': 249,

In [9]:
vocab = {'<PAD>': 0, '<UNK>': 1}
for word, freq in counter.items():
    if word not in vocab:
        vocab[word] = len(vocab)
vocab

{'<PAD>': 0,
 '<UNK>': 1,
 'go': 2,
 'until': 3,
 'jurong': 4,
 'point,': 5,
 'crazy..': 6,
 'available': 7,
 'only': 8,
 'in': 9,
 'bugis': 10,
 'n': 11,
 'great': 12,
 'world': 13,
 'la': 14,
 'e': 15,
 'buffet...': 16,
 'cine': 17,
 'there': 18,
 'got': 19,
 'amore': 20,
 'wat...': 21,
 'ok': 22,
 'lar...': 23,
 'joking': 24,
 'wif': 25,
 'u': 26,
 'oni...': 27,
 'free': 28,
 'entry': 29,
 '2': 30,
 'a': 31,
 'wkly': 32,
 'comp': 33,
 'to': 34,
 'win': 35,
 'fa': 36,
 'cup': 37,
 'final': 38,
 'tkts': 39,
 '21st': 40,
 'may': 41,
 '2005.': 42,
 'text': 43,
 '87121': 44,
 'receive': 45,
 'question(std': 46,
 'txt': 47,
 "rate)t&c's": 48,
 'apply': 49,
 "08452810075over18's": 50,
 'dun': 51,
 'say': 52,
 'so': 53,
 'early': 54,
 'hor...': 55,
 'c': 56,
 'already': 57,
 'then': 58,
 'say...': 59,
 'nah': 60,
 'i': 61,
 "don't": 62,
 'think': 63,
 'he': 64,
 'goes': 65,
 'usf,': 66,
 'lives': 67,
 'around': 68,
 'here': 69,
 'though': 70,
 'freemsg': 71,
 'hey': 72,
 'darling': 73,
 "it

In [10]:
def tokens_to_indices(tokens, vocab):
    return [vocab.get(token, vocab['<UNK>']) for token in tokens]
data['indices'] = data['tokens'].apply(lambda tokens: tokens_to_indices(tokens, vocab))
data

,label,text,tokens,indices
1,0,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point,, crazy.., available...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
2,0,Ok lar... Joking wif u oni...,"[ok, lar..., joking, wif, u, oni...]","[22, 23, 24, 25, 26, 27]"
3,1,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[28, 29, 9, 30, 31, 32, 33, 34, 35, 36, 37, 38..."
4,0,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor..., u, c, already...","[26, 51, 52, 53, 54, 55, 26, 56, 57, 58, 59]"
5,0,"Nah I don't think he goes to usf, he lives aro...","[nah, i, don't, think, he, goes, to, usf,, he,...","[60, 61, 62, 63, 64, 65, 34, 66, 64, 67, 68, 6..."
...,...,...,...,...
5568,1,This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[177, 103, 160, 439, 530, 402, 136, 484, 30, 9..."
5569,0,Will Ì_ b going to esplanade fr home?,"[will, ì_, b, going, to, esplanade, fr, home?]","[232, 300, 276, 297, 34, 12695, 1627, 13384]"
5570,0,"Pity, * was in mood for that. So...any other s...","[pity,, *, was, in, mood, for, that., so...any...","[13493, 3641, 389, 9, 10355, 89, 398, 13494, 1..."
5571,0,The guy did some bitching but I acted like i'd...,"[the, guy, did, some, bitching, but, i, acted,...","[160, 3727, 340, 85, 13496, 401, 61, 12097, 84..."


In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(len(train_data), len(test_data))

4457 1115


In [12]:
import torch

class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df.copy()
        self.df.reset_index(drop=True, inplace=True)

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        indices = self.df.loc[index, 'indices']
        label = self.df.loc[index, 'label']
        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.long)

train_dataset = SpamDataset(train_data)
test_dataset = SpamDataset(test_data)
print(train_dataset, test_dataset)

<__main__.SpamDataset object at 0x11690ac80> <__main__.SpamDataset object at 0x11690ae60>


In [13]:
from torch.utils.data import DataLoader

def collate_batch(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=vocab['<PAD>'])
    labels = torch.stack(labels)
    return texts_padded, labels

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_batch)


### 3. Model Building

Define your deep learning model's architecture.

In [16]:
# Write your code here for Model Building here

# define your Deep Learning Model here, training is in the next section.

from torch import nn
import logging
class SpamDetection(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, padding_idx):
        super(SpamDetection, self).__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        # print({"embedded": embedded})

        lstm_out, (h_n, c_n) = self.lstm(embedded)
        # print({"lstm_out": lstm_out, "h_n": h_n, "c_n": c_n})

        out = self.fc(h_n[-1])
        # print({"out": out})
        return out

### 4. Model Training

Train your model and evaluate its performance using validation data.

In [17]:
# Write your code here for Model Training here
dl_loss_value = 1

#define the iteration
import torch.optim as optim
from torch import nn

vocab_size = len(vocab)
embed_dim = 100
hidden_dim = 128
num_classes = 2
padding_idx = vocab['<PAD>']

model = SpamDetection(vocab_size, embed_dim, hidden_dim, num_classes, padding_idx)
device = "cpu"
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#create the training loop
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import confusion_matrix, classification_report

latest_pred = {"accuracy": None, "cm": None, "report": None}
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    dl_loss_value = avg_loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    all_labels = []
    all_preds = []
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=['ham', 'spam'])

    latest_pred["accuracy"] = accuracy
    latest_pred["cm"] = cm
    latest_pred["report"] = report


print("Training complete!")


Epoch 1/25, Loss: 0.3089
Epoch 2/25, Loss: 0.1999
Epoch 3/25, Loss: 0.1541
Epoch 4/25, Loss: 0.2100
Epoch 5/25, Loss: 0.1893
Epoch 6/25, Loss: 0.1162
Epoch 7/25, Loss: 0.0873
Epoch 8/25, Loss: 0.1340
Epoch 9/25, Loss: 0.0960
Epoch 10/25, Loss: 0.1433
Epoch 11/25, Loss: 0.1380
Epoch 12/25, Loss: 0.1353
Epoch 13/25, Loss: 0.2237
Epoch 14/25, Loss: 0.2941
Epoch 15/25, Loss: 0.2964
Epoch 16/25, Loss: 0.2938
Epoch 17/25, Loss: 0.2702
Epoch 18/25, Loss: 0.1764
Epoch 19/25, Loss: 0.1366
Epoch 20/25, Loss: 0.1468
Epoch 21/25, Loss: 0.2047
Epoch 22/25, Loss: 0.2022


/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Epoch 23/25, Loss: 0.3707


/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Epoch 24/25, Loss: 0.3995


/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

Epoch 25/25, Loss: 0.3829
Training complete!


/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/samuel-sqe/DEV/TEST_AREA/ai-bootcamp/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

### 5. Model Evaluation
Evaluate your model's performance on the test data using the grading scheme defined above.

In [18]:
print(f"accuracy: {latest_pred['accuracy']}")

print("confusion matrix:")
print(latest_pred['cm'])

print("classification report:")
print(latest_pred['report'])

# Write your code here for Model Evaluation here
dl_accuracy = latest_pred['accuracy']

#define the iteration

#create the training loop


accuracy: 86.54708520179372
confusion matrix:
[[965   0]
 [150   0]]
classification report:
              precision    recall  f1-score   support

         ham       0.87      1.00      0.93       965
        spam       0.00      0.00      0.00       150

    accuracy                           0.87      1115
   macro avg       0.43      0.50      0.46      1115
weighted avg       0.75      0.87      0.80      1115



In [19]:
print(dl_accuracy)
print(dl_loss_value)
print(drive_link)

86.54708520179372
0.38291910024625914
https://drive.google.com/drive/folders/1K6U077XHpD5FwrN--cFBJH6g6G2XW2rP?usp=sharing


## Submission

Once you are satisfied with the performance of your model, then you run the code block below to submit your project.


In [20]:
# Submit Method

# Do not change the code below
question_id = "01_dl_project_accuracy"
submit(student_id, name, assignment_id, str(dl_accuracy), question_id, drive_link)
question_id = "02_dl_project_loss_value"
submit(student_id, name, assignment_id, str(dl_loss_value), question_id, drive_link)

'Assignment successfully submitted'

## FIN